In [1]:
# import os

# # 设置环境变量
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# # 打印环境变量以确认设置成功
# print(os.environ.get('HF_ENDPOINT'))

# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [2]:
import json
import random
import re
import os
from datasets import load_dataset
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# ==========================================
# 1. 配置 API  https://bailian.console.aliyun.com/
# ==========================================
API_KEY = "sk-"  # 注意：在生产环境中请勿硬编码，建议使用 os.getenv
MODEL_ID = "qwen3-max"

client = OpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=API_KEY,
)

In [4]:
# ==========================================
# 2. 数据准备 (加载、采样、打乱) - TF Binding 版本
# ==========================================
print("Loading dataset...")
try:
    dataset = load_dataset('dnagpt/dna_transcription_factor_prediction')
    # 假设你的本地数据集路径
    # local_dataset_path = "./dna_transcription_factor_prediction"
    
    # # 加载数据集
    # # 如果是本地文件夹且包含 arrow/json/csv，或者是一个 dataset loading script
    # dataset = load_dataset(
    #     local_dataset_path, 
    #     trust_remote_code=True
    # )
    
    # 获取训练集
    ds = dataset['train']
    
    # ⚠️ 关键修改：检查数据集的列名
    # 通常列名可能是 'sequence', 'seq', 'dna', 'sentence' 等
    # 这里假设列名就是你描述的 'sequence'
    print(f"Dataset columns: {ds.column_names}") 
    
    # 1. 分离数据 (Label 0 和 Label 1)
    # 注意：确保这里使用的是正确的键名，这里假设是 'label' 和 'sequence'
    data_label_0 = [item for item in ds if item['label'] == 0]
    data_label_1 = [item for item in ds if item['label'] == 1]
    
    # 2. 随机采样 (各50个，凑100个测试)
    random.seed(42)
    sample_num = 50 
    
    # 防止数据不够报错，加个 min 取值
    real_sample_num_0 = min(sample_num, len(data_label_0))
    real_sample_num_1 = min(sample_num, len(data_label_1))
    
    sampled_0 = random.sample(data_label_0, real_sample_num_0)
    sampled_1 = random.sample(data_label_1, real_sample_num_1)
    
    # 3. 合并并打乱
    combined_data = sampled_0 + sampled_1
    random.shuffle(combined_data)
    
    print(f"Data prepared: {len(combined_data)} sequences (Positive: {len(sampled_1)}, Negative: {len(sampled_0)})")

except Exception as e:
    print(f"Error loading dataset: {e}")
    # 打印详细错误方便调试
    import traceback
    traceback.print_exc()
    exit()

# ==========================================
# 构建用于 Prompt 的 JSON List
# ==========================================
prompt_data_list = []
id_to_ground_truth = {}

for idx, item in enumerate(combined_data, 1):
    # ⚠️ 关键修改：单序列任务，JSON 里不需要 seq_a, seq_b
    # 只需要一个 "sequence" 字段
    
    # 容错：有些数据集把序列叫 'sentence'，有些叫 'sequence'，有些叫 'seq'
    # 这里的逻辑是优先找 'sequence'，找不到就找 'sentence'
    seq_content = item.get('sequence', item.get('sentence', item.get('seq', '')))
    
    prompt_data_list.append({
        "id": idx,
        "sequence": seq_content # 👈 只要这一条
    })
    
    # 记录真值
    id_to_ground_truth[idx] = item['label']

print("Prompt data constructed.")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset './dna_transcription_factor_prediction' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading dataset...
Dataset columns: ['sequence', 'label']
Data prepared: 100 sequences (Positive: 50, Negative: 50)
Prompt data constructed.


In [5]:
# # ==========================================
# # 3. 存储数据到文件 (包含 Label)
# # ==========================================
# import json

# # 定义文件名
# output_filename = "prompt_data_list.json"

# print(f"Saving {len(prompt_data_list)} items to {output_filename}...")

# # --- 关键修改：在保存前，把 label 从 id_to_ground_truth 中合并进去 ---
# # 这一步不需要动前面的代码，直接利用已有的 id_to_ground_truth 字典
# for item in prompt_data_list:
#     item_id = item['id']
#     if item_id in id_to_ground_truth:
#         item['label'] = id_to_ground_truth[item_id]

# # 写入文件
# with open(output_filename, "w", encoding="utf-8") as f:
#     # indent=2 让文件格式化（有缩进），方便人类阅读
#     # ensure_ascii=False 防止中文（如果有的话）被转义成乱码
#     json.dump(prompt_data_list, f, indent=2, ensure_ascii=False)

# print("Save complete (with labels).")

In [6]:
# ==========================================
# 3. 构建 Prompt (TF Binding - 信号检测版本)
# ==========================================

# System Prompt: 将 TF Binding 类比为“在噪音中寻找特定指令”
system_prompt = """You are an expert in Genomic Syntax and Pattern Recognition.

1. The Task:
You are analyzing short DNA segments to determine if they function as a "Regulatory Switch" (Transcription Factor Binding Site).

2. The Analogy (NLP Equivalent):
* Imagine looking at a stream of text. Most of it is random jibberish or filler text (Background Genomic Noise).
* However, some segments contain a specific "Password" or "Trigger Phrase" (The Motif) that activates a system.
* Your job is to classify whether the text contains this "Active Trigger".

3. The Definition:
* **Label 1 (Positive/Bound)**: The sequence acts as a **"Valid Command"**. It contains a specific, non-random pattern (motif) that allows a protein to dock and initiate transcription. It looks "purposeful" and "conserved".
* **Label 0 (Negative/Unbound)**: The sequence is **"Passive Noise"**. It lacks the specific structural syntax required for binding. It looks like random background text.

4. Output Requirements:
* I will provide a JSON list of DNA sequences.
* You must return a RAW JSON object containing a list of results.
* The format must be strictly: `[{"id": 1, "prediction": 1}, {"id": 2, "prediction": 0}, ...]`
* Do NOT provide explanations. Just the JSON array.
"""



# User Prompt: 强调寻找“显著性特征”
user_prompt = f"""Here is the list of DNA sequences to classify.

Determine if each sequence is a **Functional Binding Site (1)** or **Non-Functional Background (0)**.
Use your intuition about "sequence complexity", "motif occurence", and "biological signal intensity".

Data:
{json.dumps(prompt_data_list, indent=2)}
"""

In [7]:
# ==========================================
# 4. 调用 Volcengine API
# ==========================================
print("-" * 30)
print(f"Calling Volcengine Model: {MODEL_ID}...")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1, # 低温以保证输出格式稳定
        top_p=0.9,
    )
    
    full_content = response.choices[0].message.content.strip()
    print("Response received.")
    # 打印前200个字符用于调试
    print(f"Response snippet: {full_content[:200]}...")

except Exception as e:
    print(f"API Call Failed: {e}")
    full_content = ""

------------------------------
Calling Volcengine Model: qwen3-max...
Response received.
Response snippet: [{"id": 1, "prediction": 1}, {"id": 2, "prediction": 0}, {"id": 3, "prediction": 0}, {"id": 4, "prediction": 1}, {"id": 5, "prediction": 1}, {"id": 6, "prediction": 1}, {"id": 7, "prediction": 0}, {"i...


In [8]:
# ==========================================
# 5. 解析结果与评估 (TF Binding 0/1 版本)
# ==========================================
import re
import json
from sklearn.metrics import accuracy_score, classification_report

def parse_llm_json(text):
    """提取并解析 JSON"""
    try:
        # 优先找代码块
        code_block = re.search(r"```json\s*(\[.*?\])\s*```", text, re.DOTALL)
        if code_block:
            return json.loads(code_block.group(1))
        
        # 找不到代码块就找方括号
        match = re.search(r"\[.*\]", text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
            
        return json.loads(text)
    except:
        return [] 

# 解析模型输出
predictions_list = parse_llm_json(full_content)

y_true = []
y_pred = []

print("-" * 30)
if not predictions_list:
    print("Failed to parse JSON from model response.")
else:
    print(f"Parsed {len(predictions_list)} predictions.")
    
    for item in predictions_list:
        p_id = item.get('id')
        p_val = item.get('prediction') # 这里拿到的应该是 0 或 1
        
        # 简单的校验：ID 存在且 预测值是 0 或 1
        if p_id in id_to_ground_truth and p_val in [0, 1]:
            y_true.append(id_to_ground_truth[p_id])
            y_pred.append(int(p_val)) # 确保转成整数
        else:
            # 调试用，看看是不是模型没按套路出牌
            pass 

    # 输出最终指标
    if y_true:
        acc = accuracy_score(y_true, y_pred)
        print(f"\nFinal Accuracy: {acc:.2%}")
        print("\nClassification Report:")
        # 这里的名字改成了 Unbound 和 Bound
        print(classification_report(y_true, y_pred, target_names=["Unbound (0)", "Bound (1)"]))
        
        result_log = {
            "accuracy": acc,
            "predictions": predictions_list
        }
        print(acc)
    else:
        print("No valid matching IDs found.")

------------------------------
Parsed 100 predictions.

Final Accuracy: 62.00%

Classification Report:
              precision    recall  f1-score   support

 Unbound (0)       0.62      0.64      0.63        50
   Bound (1)       0.62      0.60      0.61        50

    accuracy                           0.62       100
   macro avg       0.62      0.62      0.62       100
weighted avg       0.62      0.62      0.62       100

0.62
